#### install pandas

In [ ]:

! pip3 install pandas


In [ ]:
! pip3 show pandas

In [ ]:
!pip3 install tabula-py


In [ ]:
!pip3 install PyMuPDF pandas

In [ ]:
!pip3 install psycopg2-binary

In [ ]:
import fitz  # PyMuPDF
import re
import pandas as pd
import psycopg2
from psycopg2 import sql
import datetime
import requests

# Function to extract text from PDF and remove text before ***
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    
    # Remove text before ***
    if "***" in text:
        text = text.split("***", 1)[1]

    # Remove text after www.scb.co.th
    if "Total amount" in text:
        text = text.split("Total amount", 1)[0] 
    
    return text

# Function to extract date and time pattern (DD/MM/YY HH:MM) excluding full date range (DD/MM/YYYY-DD/MM/YYYY)
def get_date_time(text):
    # Regular expression to match date and time pattern (DD/MM/YY HH:MM) but not full date range (DD/MM/YYYY-DD/MM/YYYY)
    date_time_pattern = re.compile(r'\b\d{2}/\d{2}/\d{2}\s+\d{2}:\d{2}\b(?!-\d{2}/\d{2}/\d{4})')
    matches = date_time_pattern.findall(text)
    return matches

# Function to extract Code/Channel and the number after it
def get_code_channel_and_amount(text):
    # Regular expression to match Code/Channel and the number after it
    code_channel_pattern = re.compile(r'\b([A-Z0-9]+/[A-Z]+)\b\s+([\d,.]+)')
    matches = code_channel_pattern.findall(text)
    return matches

# Function to extract DESC/NOTE
def get_desc_note(text):
    # Regular expression to match long text with at least 13 characters including spaces, parentheses, and forward slash
    desc_note_pattern = re.compile(r'\b(?=[A-Za-z])[\w\s()/]{13,}\b')
    matches = desc_note_pattern.findall(text)
    return matches

# Main function
def main(pdf_path):
    text = extract_text_from_pdf(pdf_path)
    date_times = [pd.to_datetime(dt, format='%d/%m/%y %H:%M').strftime('%Y-%m-%d %H:%M:%S.%f')[:-3] for dt in get_date_time(text)]
    code_channel_and_amount = get_code_channel_and_amount(text)
    desc_note = get_desc_note(text)

    # Create data table
    data = {'date': date_times, 'code': [x[0] for x in code_channel_and_amount], 'amount': [float(x[1].replace(',', '')) for x in code_channel_and_amount], 'desc': desc_note}
    pdfDetect = pd.DataFrame(data)
    return pdfDetect

# Replace 'AcctSt.pdf' with your PDF file path
# pdf_path = '/Volumes/LACIES/Flexi-React-Expo/python/pdf/AcctSt2.pdf'
# pdfDetect = main(pdf_path)
response = requests.get('http://localhost:3000/pdf-path')
pdf_path = response.json().get('pdfPath')

if pdf_path:
    pdfDetect = main(pdf_path)

# exclude row that has contain X1
pdfDetect = pdfDetect[~pdfDetect['code'].str.contains('X1')]
print(pdfDetect)


# Connect to your PostgreSQL database
conn = psycopg2.connect(
    dbname="prismatypedb",
    user="postgres",
    password="123456",
    host="127.0.0.1",
    port="9649",
)

# print("Database opened successfully")

# Open a cursor to perform database operations
cur = conn.cursor()

# Show data in Expense table
cur.execute("SELECT * FROM flexidb.\"Expense\";")
for record in cur.fetchall():
    print("Expense : ", record)

# Insert data into Expense table
for index, row in pdfDetect.iterrows():
    cur.execute(
        sql.SQL("INSERT INTO flexidb.\"Expense\" (date, code, amount, \"desc\", \"businessAcc\", \"memberId\", \"updatedAt\") VALUES ({}, {}, {}, {}, {}, {}, {});").format(
            sql.Literal(row['date']),
            sql.Literal(row['code']),
            sql.Literal(row['amount']),
            sql.Literal(row['desc']),
            sql.Literal(4),
            sql.Literal('1869d1d1-c66a-4c64-b3a9-e6589b368978'),
            sql.Literal(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        )
    )
    print("Successfully inserted record")
# Commit the transaction
conn.commit()


# # Close the cursor and connection
cur.close()
conn.close()

                      date     code   amount                        desc
0  2024-02-02 15:42:00.000  X2/ENET    42.68         SCB x1937 นางสาว ธน
2  2024-02-06 08:16:00.000  X2/ENET  1151.00         SCB x1937 นางสาว ธน
4  2024-02-12 15:47:00.000  X2/ENET  1300.00         SCB x1937 นางสาว ธน
5  2024-02-13 09:55:00.000  X2/ENET    19.00  PromptPay x2696 นางสาว ณณว
6  2024-02-16 12:31:00.000  X2/ENET    71.00         SCB x1937 นางสาว ธน
8  2024-02-23 15:17:00.000  X2/ENET    25.00          KERRY EXPRESS GBKK
Expense :  (datetime.datetime(2024, 2, 2, 15, 42), datetime.datetime(2025, 3, 4, 8, 38, 32, 217000), datetime.datetime(2025, 3, 4, 15, 38, 32), 94, 43, None, None, 4, '1869d1d1-c66a-4c64-b3a9-e6589b368978', False, None, None, 'X2/ENET', 'SCB x1937 นางสาว ธน')
Expense :  (datetime.datetime(2024, 2, 6, 8, 16), datetime.datetime(2025, 3, 4, 8, 38, 32, 217000), datetime.datetime(2025, 3, 4, 15, 38, 32), 95, 1151, None, None, 4, '1869d1d1-c66a-4c64-b3a9-e6589b368978', False, None, None, 'X

In [ ]:
from flask import Flask, request, jsonify
import fitz  # PyMuPDF
import re
import pandas as pd
import psycopg2
from psycopg2 import sql
import datetime

app = Flask(__name__)

# Function to extract text from PDF and remove text before ***
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    
    # Remove text before ***
    if "***" in text:
        text = text.split("***", 1)[1]

    # Remove text after www.scb.co.th
    if "Total amount" in text:
        text = text.split("Total amount", 1)[0] 
    
    return text

# Function to extract date and time pattern (DD/MM/YY HH:MM) excluding full date range (DD/MM/YYYY-DD/MM/YYYY)
def get_date_time(text):
    # Regular expression to match date and time pattern (DD/MM/YY HH:MM) but not full date range (DD/MM/YYYY-DD/MM/YYYY)
    date_time_pattern = re.compile(r'\b\d{2}/\d{2}/\d{2}\s+\d{2}:\d{2}\b(?!-\d{2}/\d{2}/\d{4})')
    matches = date_time_pattern.findall(text)
    return matches

# Function to extract Code/Channel and the number after it
def get_code_channel_and_amount(text):
    # Regular expression to match Code/Channel and the number after it
    code_channel_pattern = re.compile(r'\b([A-Z0-9]+/[A-Z]+)\b\s+([\d,.]+)')
    matches = code_channel_pattern.findall(text)
    return matches

# Function to extract DESC/NOTE
def get_desc_note(text):
    # Regular expression to match long text with at least 13 characters including spaces, parentheses, and forward slash
    desc_note_pattern = re.compile(r'\b(?=[A-Za-z])[\w\s()/]{13,}\b')
    matches = desc_note_pattern.findall(text)
    return matches

# Main function
def main(pdf_path):
    text = extract_text_from_pdf(pdf_path)
    date_times = [pd.to_datetime(dt, format='%d/%m/%y %H:%M').strftime('%Y-%m-%d %H:%M:%S.%f')[:-3] for dt in get_date_time(text)]
    code_channel_and_amount = get_code_channel_and_amount(text)
    desc_note = get_desc_note(text)

    # Create data table
    data = {'date': date_times, 'code': [x[0] for x in code_channel_and_amount], 'amount': [float(x[1].replace(',', '')) for x in code_channel_and_amount], 'desc': desc_note}
    pdfDetect = pd.DataFrame(data)
    return pdfDetect

@app.route('/pdf-path', methods=['POST'])
def receive_pdf_path():
    data = request.get_json()
    pdf_path = data.get('pdfPath')
    if not pdf_path:
        return jsonify({'error': 'PDF path is required'}), 400

    pdfDetect = main(pdf_path)

    # exclude row that has contain X1
    pdfDetect = pdfDetect[~pdfDetect['code'].str.contains('X1')]
    print(pdfDetect)

    # Connect to your PostgreSQL database
    conn = psycopg2.connect(
        dbname="prismatypedb",
        user="postgres",
        password="123456",
        host="127.0.0.1",
        port="9649",
    )

    # Open a cursor to perform database operations
    cur = conn.cursor()

    # Insert data into Expense table
    for index, row in pdfDetect.iterrows():
        cur.execute(
            sql.SQL("INSERT INTO flexidb.\"Expense\" (date, code, amount, \"desc\", \"businessAcc\", \"memberId\", \"updatedAt\") VALUES ({}, {}, {}, {}, {}, {}, {});").format(
                sql.Literal(row['date']),
                sql.Literal(row['code']),
                sql.Literal(row['amount']),
                sql.Literal(row['desc']),
                sql.Literal(4),
                sql.Literal('1869d1d1-c66a-4c64-b3a9-e6589b368978'),
                sql.Literal(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
            )
        )
        print("Successfully inserted record")
    # Commit the transaction
    conn.commit()

    # Close the cursor and connection
    cur.close()
    conn.close()

    return jsonify({'message': 'PDF path processed successfully'})

if __name__ == '__main__':
    app.run(debug=True)